In [4]:
import pandas as pd
import numpy as np
import nltk
import re
import string
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

#Read file of joined Yelp Business & Review data
df = pd.read_csv('../Output_Folder/Santa_Barbara_Yelps.csv')
df.head(5)

,business_id,name,address,city,state,postal_code,review_count,review_id,stars,text
0,gebiRewfieSdtt17PTW6Zg,Hibachi Steak House & Sushi Bar,502 State St,Santa Barbara,CA,93101,488,pUycOfUwM8vqX7KjRRhUEA,3.0,Had a party of 6 here for hibachi. Our waitres...
1,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101,389,L0jv8c2FbpWSlfNC6bbUEA,5.0,What a great addition to the Funk Zone! Grab ...
2,qO9dNNIvNbCBd8ZgjxMxgQ,HOME Santa Barbara,14 Parker Way,Santa Barbara,CA,93101,13,2u5Skti5mZam_-XTKPelvA,5.0,"Farmhouse, rustic, chic.Helpful staff with gre..."
3,SZU9c8V2GuREDN5KgyHFJw,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,2404,4zopEEPqfwm-c_FNpeHZYw,5.0,We were a bit weary about trying the Shellfish...
4,YbnJYHNp_fHbI-hcFg48vQ,Santa Barbara Adventure Company,32 E Haley St,Santa Barbara,CA,93101,195,HuWEnZr7-0HveaqXxyywgg,5.0,The kayaking tour at the Santa Cruz Island was...


In [11]:
df.to_excel('../Output_Folder/EX_S_B_Y.xlsx')

In [5]:
#Read file of joined Yelp Business & Review data
df2 = pd.read_csv('../Output_Folder/pre_process.csv')
df2.head(5)

,Unnamed: 0,text,stars,label,pre_process
0,0,I usually stay away from restaurants on piers ...,1,0,usually stay away restaurant pier since touris...
1,1,Meh not worth the $5 per person fee. Confusing...,2,0,meh worth per person fee confusing layout bori...
2,2,Why did they offer to place a pick up order if...,2,0,offer place pick order going right ordered won...
3,3,"Super slow service, salads are not very good a...",2,0,super slow service salad good really priced re...
4,4,I would not advise staying at this hotel. I ca...,1,0,would advise staying hotel called speak manage...


In [8]:
df.dtypes

business_id      object
name             object
address          object
city             object
state            object
postal_code       int64
review_count      int64
review_id        object
stars           float64
text             object
dtype: object

In [15]:
df['stars']=df['stars'].astype(int) #convert stars column to int

df=df[df['stars']!=3] #Drop 3-star reviews

In [16]:
df

,business_id,name,address,city,state,postal_code,review_count,review_id,stars,text
1,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101,389,L0jv8c2FbpWSlfNC6bbUEA,5.0,What a great addition to the Funk Zone! Grab ...
2,qO9dNNIvNbCBd8ZgjxMxgQ,HOME Santa Barbara,14 Parker Way,Santa Barbara,CA,93101,13,2u5Skti5mZam_-XTKPelvA,5.0,"Farmhouse, rustic, chic.Helpful staff with gre..."
3,SZU9c8V2GuREDN5KgyHFJw,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,2404,4zopEEPqfwm-c_FNpeHZYw,5.0,We were a bit weary about trying the Shellfish...
4,YbnJYHNp_fHbI-hcFg48vQ,Santa Barbara Adventure Company,32 E Haley St,Santa Barbara,CA,93101,195,HuWEnZr7-0HveaqXxyywgg,5.0,The kayaking tour at the Santa Cruz Island was...
5,6JFTijOMHB46yBoyVOjPCA,Santa Barbara County Courthouse,1100 Anacapa St,Santa Barbara,CA,93101,251,GK5JEpizD-nDGXsJUYVLTg,4.0,The court house is very old and the art is gre...
...,...,...,...,...,...,...,...,...,...,...
481,Vz1QSF4nurQv2-dok55jZg,"Howard Gross, MD","1722 State St, Ste 201",Santa Barbara,CA,93101,16,NaN,NaN,NaN
482,jNWePdDkWzwt4oQ5UZulIg,Faitell Attractions,619 State St,Santa Barbara,CA,93101,10,NaN,NaN,NaN
483,8-ZExybRuyyXZf6aESgc3g,Paseo Nuevo Shopping Center,651 Paseo Nuevo,Santa Barbara,CA,93101,125,NaN,NaN,NaN
484,EHF8hnwsWES8lmedq5eqqA,Skivvies Esthetics,"11 W. de La Guerra St, Ste B",Santa Barbara,CA,93101,10,NaN,NaN,NaN
